In [1]:
import os
import math
import copy
import time
import random
import pickle
import shutil
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from config_utils.config_utils import *
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import Dataset


os.environ["CUDA_VISIBLE_DEVICES"] = "3"  
torch.cuda.device_count()

1

In [2]:
# convert json to pickle

# json2pickle('./versions/production-v3/checkpoint/config.json', './versions/production-v3/checkpoint/config.pkl')


In [3]:
# load config, if exists
config = None
folder_path = './versions/production-v3/checkpoint/'

if os.path.isfile(folder_path + 'config.pkl'):
    print('loading the config file..')
    with open(folder_path + 'config.pkl', 'rb') as fp:
        config = pickle.load(fp)

loading the config file..


In [4]:
prev_data_pth = './versions/production-v2/training_data/clean_100_rest_0_no_feedback_base_pe_data_plus_amazon_gamma_plus_new_pe_otc_catalog.pkl'
delta_data_pth = './versions/production-v3/training_data/delta_data_v3_pe_catalog.csv'

tdata = pickle.load(open(prev_data_pth, "rb"), encoding="latin1")
data = tdata['training_data']

new = pd.read_csv(delta_data_pth)[['input', 'output']].values.tolist()

data += new
print('training data size - {l} rows'.format(l=str(len(data))))

training data size - 6948142 rows


In [5]:
# last trained on clean_100_rest_0_no_feedback_base_pe_data.pkl
# training_data_path = './training_data/clean_100_rest_0_no_feedback_base_pe_data.pkl'
# training_data_path = './training_data/clean_100_rest_0_no_feedback_base_pe_data_plus_amazon.pkl'

In [6]:
# training_data_path = './training_data/clean_100_rest_0_no_feedback_base_pe_data_plus_amazon_stuart_plus_new_pe_otc_catalog_plus_nla_plus_feedback.pkl'
# kdata = pickle.load(open(training_data_path, "rb"), encoding="latin1")
# kdata = kdata['training_data']
# data.extend(kdata)

In [5]:
import re

for d in tqdm(data):
    try:
        d[1] = re.sub('ALL ONE|ALL NATURALS|ALL NATURAL|ALL PURPOSE|ALL OTHER|ALL SKIN|MADE IN INDIA|ALL IN ONE|TOP BRANDS|POPULAR CATEGORIES', '', d[1])
        d[1] = re.sub(r' +', ' ', d[1]).strip()

        d[0] = re.sub('ALL ONE|ALL NATURALS|ALL NATURAL|ALL PURPOSE|ALL OTHER|ALL SKIN|MADE IN INDIA|ALL IN ONE|TOP BRANDS|POPULAR CATEGORIES', '', d[0])
        d[0] = re.sub(r' +', ' ', d[0]).strip()
    except:
        data.remove(d)
        print(d)

 98%|█████████▊| 6802892/6948142 [01:01<00:04, 32501.56it/s] 

[nan, 'LUXOR NA PEN SANITIZER SPRAY REFILL BOTTLE 10 ML 20 ML CLASSIC SPRAY SURFACE SANITIZERS HOME HYGEINE SURFACE SANITIZATION KIT HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']
[nan, 'LUXOR NA DISINFECTANT SPRAY CITRUS FRESH 241ML SPRAY DISINFECTANTS HOME HYGEINE SURFACE SANITIZATION DISINFECTANT BOTTLE HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']


 98%|█████████▊| 6818947/6948142 [01:01<00:03, 35635.72it/s]

[nan, 'VITROVEA MEGACURC NA CURCUMIN TRIPLE STRENGTH FISH OIL W/10X STRENGTH BOSWELLIA 60 TABLETS CAPSULE IMMUNITY BOOSTER IMMUNITY BOOSTER IMMUNITY SUPPLEMENT BOTTLE FOOD NUTRITION SPECIALITY SUPPLEMENTS FOOD INNOVERTUS NUTRITION TECHNOLOGIES']
[nan, 'LUXOR NA PEN SANITIZER SPRAY TWIN CLASSIC 10ML X2 SPRAY SURFACE SANITIZERS HOME HYGEINE SURFACE SANITIZATION KIT HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']
[nan, 'LUXOR NA PEN SANITIZER SPRAY TWIN LEMON ZEST 10ML X2 SPRAY SURFACE SANITIZERS HOME HYGEINE SURFACE SANITIZATION KIT HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']
[nan, 'LUXOR NA POCKET DISINFECTANT SPRAY PINE FRESH 54 ML 18 ML X3 SPRAY DISINFECTANTS HOME HYGEINE SURFACE SANITIZATION DISINFECTANT BOTTLE HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']


 99%|█████████▉| 6866947/6948142 [01:03<00:01, 47957.29it/s]

...

[nan, 'LUXOR NA DISINFECTANT SPRAY PINE FRESH 95ML SPRAY DISINFECTANTS HOME HYGEINE SURFACE SANITIZATION DISINFECTANT BOTTLE HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']


100%|█████████▉| 6943740/6948142 [01:05<00:00, 20214.45it/s]

[nan, 'LUXOR NA PEN SANITIZER SPRAY REFILL BOTTLE 10 ML 20 ML LEMON ZEST SPRAY SURFACE SANITIZERS HOME HYGEINE SURFACE SANITIZATION KIT HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']
[nan, 'LUXOR NA MULTIPUPOSE DISINFECTANT CLEANER 500ML LIQUID DISINFECTANTS DISINFECTANT BOTTLE HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']
[nan, 'LUXOR NA DISINFECTANT SPRAY CITRUS FRESH 95ML SPRAY DISINFECTANTS HOME HYGEINE DISINFECTANT SURFACE SANITIZATION BOTTLE HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']
[nan, 'LUXOR NA MOBILE DISINFECTANT CLEANER 25ML TRAVEL POUCH LIQUID DISINFECTANTS DISINFECTANT BOTTLE HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']
[nan, 'LUXOR NA DISINFECTANT SPRAY PINE FRESH 241ML SPRAY DISINFECTANT HOME HYGEINE SURFACE SANITIZATION DISINFECTANT BOTTLE HOME CARE HOME HYGIENE ESSENTIALS SANITIZER WIPES LUXOR NA']


100%|█████████▉| 6948124/6948142 [01:06<00:00, 104045.47it/s]

[nan, 'HAEAL GARCINIA 60 CAPSULES 400G CAPSULE SUPPLEMENT MULTIVITAMINS AND MULTIMINERALS AARSHAVEDA WELLNESS PVT LTD.']
[nan, 'HAEAL SPIRULINA 60 CAPSULES 500G CAPSULE SUPPLEMENT MULTIVITAMINS AND MULTIMINERALS AARSHAVEDA WELLNESS PVT LTD.']
[nan, 'NATXTRA CURQMEG 3 SUPPORTS CARDIAC HEALTH 1X 60 CAPSULES CAPSULE BLOOD CIRCULATION ENHANCERS HEART AND BLOOD VESSELS CARE SYNTHITE INDUSTRIES PVT.LTD.']


In [6]:
if not config:
    max_word_len = 0
    max_sent_len = 0
    max_output_len = 0

    for inp, out in data:
        try:
            inp, out = str(inp), str(out)
            max_word_len = max(max_word_len, max([len(word) for word in inp.split()]))
            max_sent_len = max(max_sent_len, len(inp.split()))
            max_output_len = max(max_output_len, len(out))
        except:
            print([inp, out])

    print(
        "original parameters - max_word_len = {}, max_sent_len = {}, max_output_len = {}".format(
            max_word_len, max_sent_len, max_output_len
        )
    )

    max_word_len = min(28, max_word_len)
    max_sent_len = min(15, max_sent_len)
    max_output_len = min(200, max_output_len)

    print(
        "updated parameters - max_word_len = {}, max_sent_len = {}, max_output_len = {}".format(
            max_word_len, max_sent_len, max_output_len
        )
    )

else:
    print("loading params from config file..")

    max_word_len = config["max_word_len"]
    max_sent_len = config["max_sent_len"]
    max_output_len = config["max_output_len"]

    print(
        "parameters: max_word_len = {}, max_sent_len = {}, max_output_len = {}".format(
            max_word_len, max_sent_len, max_output_len
        )
    )

loading params from config file..
parameters: max_word_len = 28, max_sent_len = 15, max_output_len = 200


In [7]:
data = pd.DataFrame(data, columns=['in', 'out'])

data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

data = data.sample(frac=1)
data = data.apply(lambda x: x.astype(str) \
                                    .str.upper() \
                                    .str.replace(r'[^A-Z0-9\.%/\s]', '') \
                                    .str.replace(r' +', ' ') \
                                    .str.strip())

data['in'] = [' '.join([i[:max_word_len] for i in v.split()[:max_sent_len]]) for v in data['in']]
data['out'] = [v[:max_output_len-2] for v in data['out']]
    
data = data[data['in'] != '']
data = data[data['in'] != 'nan']
data = data[data['in'] != 'NAN']

data

<ipython-input-7-0d60b0366c62>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.apply(lambda x: x.astype(str) \


,in,out
6038784,VITAMIN B6 VITAMIN B5 MODI MUNDI PHARMA,GRO ACTIV CHOCOLATE FLAVOUR BOX OF 400GM POWDE...
378154,GRITA 200 CAPSULE,GRITA 200MG CAP 4S ITRACONAZOLE GRACIERA PHARM...
6601409,DHANYAKA CORIANDRUM ALOE VERA SAINDHAVA,K G TONE FORTE BOTTLE OF 100ML SYRUP SAUNF CUM...
531341,ITANOL 200MG 10CAP,ITANOL 200MG CAP 10S ITRACONAZOLE GALCARE PHAR...
6694911,FIROCOD PLUS OMEGA 3 VITAMIN B12,FIROCOD PLUS STRIP OF 10 TABLETS OMEGA 3 FATTY...
...,...,...
1155866,ONABET SD LOTION SERTACONAZOLE MOMETASONE GLEN...,ONABET SD LOTION SERTACONAZOLE MOMETASONE GLEN...
3533854,MILLESIMAL POTENCIES,SBL CRATAEGUS OXYACANTHA 0/17 LM
2305486,12 ENTER CHYMUTRIP C.TABS X,CHYMUTRIP TAB CHYMOTRYPSIN TRYPSIN BOMBAY TABL...
199070,WEIGHT MANAGEMENT NATUREAL GREEN TEA 800MG CAP...,WEIGHT MANAGEMENT NATUREAL GREEN TEA 800MG CAP...


In [8]:
class CDataset(Dataset):
    def __init__(self, data, config=None):
        self.data = data

        if config is None:
            self.max_sent_len = max(
                [len(sent.split()) for sent in self.data["in"].values]
            )
            self.max_output_len = (
                max([len(sent) for sent in self.data["out"].values]) + 2
            )
            self.max_word_len = max(
                [len(word) for sent in self.data["in"].values for word in sent.split()]
            )

            self.char2idx, self.idx2char = self.char2index(
                set(pd.concat([data["in"], data["out"]]))
            )

        else:
            self.max_word_len = config["max_word_len"]
            self.max_sent_len = config["max_sent_len"]
            self.max_output_len = config["max_output_len"]

            self.char2idx = config["char2idx"]
            self.idx2char = {v: k for k, v in self.char2idx.items()}

    def __len__(self):
        return len(self.data)

    def char2index(self, data=[]):
        data = set([char for sent in data for char in sent])

        char2idx = {c: i + 3 for i, c in enumerate(data)}
        char2idx.update({"<PAD>": 0, "<SOS>": 1, "<EOS>": 2})

        idx2char = {v: k for k, v in char2idx.items()}
        return char2idx, idx2char

    def __getitem__(self, idx):
        # output
        label = np.asarray(
            [self.char2idx["<SOS>"]]
            + [self.char2idx[x] for x in self.data["out"].values[idx]]
            + [self.char2idx["<EOS>"]]
        )
        label = torch.LongTensor(
            np.pad(
                label,
                (self.char2idx["<PAD>"], self.max_output_len - len(label)),
                "constant",
            )
        )

        # word input and padding
        sentence = [
            [self.char2idx[char] for char in word]
            for word in self.data["in"].values[idx].split()
        ]

        sentence = [
            list(
                np.pad(
                    s, (self.char2idx["<PAD>"], self.max_word_len - len(s)), "constant"
                )
            )
            for s in sentence
        ]

        # sentence input padding
        sentence += [[0] * self.max_word_len] * (self.max_sent_len - len(sentence))
        sentence = torch.LongTensor(sentence)
        return sentence, label


In [9]:
from torch.utils.data import DataLoader, random_split

# 425
BATCH_SIZE = 500
cdata = CDataset(data, config)

# train test split
train_split_ratio = 0.999999
train_split = int(train_split_ratio * len(cdata))
test_split = len(cdata) - train_split

cdata_train, cdata_test = random_split(cdata, [train_split, test_split])

# train dataloader
train_dataloader_args =  dict(shuffle=True, batch_size=BATCH_SIZE, num_workers=0)
trainloader = DataLoader(cdata_train, pin_memory=True, **train_dataloader_args)

# test dataloader
test_dataloader_args =  dict(shuffle=True, batch_size=BATCH_SIZE, num_workers=0)
testloader = DataLoader(cdata_test, pin_memory=True, **test_dataloader_args)

In [12]:
# save config
if not os.path.isfile(folder_path + 'checkpoint/scratch/config.pkl') :
    config = dict()
    
    # aug_avail and word_percentages are the training data configuration and rest are model configuration
    config.update({'max_word_len':cdata.max_word_len, 
                   'max_sent_len':cdata.max_sent_len, 
                   'max_output_len':cdata.max_output_len, 
                   'char2idx':cdata.char2idx, 
                   'aug_avail': tdata['aug_avail'], 
                   'word_percentages': tdata['word_percentages'],
                  })
    
    with open(folder_path + 'checkpoint/scratch/config.pkl', 'wb') as fp:
        pickle.dump(config, fp)

In [13]:
class TextCNN(nn.Module):
    def __init__(self, word_emb, sent_emb, kernel_sizes, dropout=0.1):
        super(TextCNN, self).__init__()

        sent_emb = int(sent_emb / len(kernel_sizes))

        # three different convolutional layers
        self.convs = nn.ModuleList([nn.Conv1d(word_emb, sent_emb, k) for k in kernel_sizes])
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def conv_and_max_pool(x, conv):
        return conv(x).permute(0, 2, 1).max(1)[0]

    def forward(self, inputs, src_mask):
        mask = src_mask.squeeze(1).int()
        inputs = inputs * mask.unsqueeze(-1).expand(inputs.size())

        inputs = inputs.permute(0, 2, 1)

        x = [self.conv_and_max_pool(inputs, k) for k in self.convs]

        return self.dropout(torch.cat(x, 1))


class Embedding(nn.Module):
    def __init__(self, vocab_size, dimension):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, dimension, padding_idx=0)
        self.dimension = dimension

    def forward(self, input_vec):
        return self.embedding(input_vec) * math.sqrt(self.dimension)


class EncEmbedding(nn.Module):
    def __init__(self, vocab_size, char_emb_dim_size, word_emb_dim_size, kernel_sizes, dropout=0.05):
        super().__init__()
        self.K = kernel_sizes
        dim = int(word_emb_dim_size / len(kernel_sizes))

        self.embedding = Embedding(vocab_size, char_emb_dim_size)
        self.convs = nn.ModuleList([nn.Conv2d(char_emb_dim_size, dim, kernel_size=(1,k)) for k in kernel_sizes])
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def convolve(conv, x):
        return conv(x).max(-1)[0].transpose(1,2)

    def forward(self, x):
        x = self.embedding(x).transpose(2,3).transpose(1,2)
        x = [self.convolve(conv, x) for conv in self.convs]
        return torch.cat(x, -1)


class PositionalEncoding(nn.Module):
    def __init__(self, max_seq_len, dimension, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        positional_enc = torch.zeros(max_seq_len, dimension)

        den = torch.pow(
            10000, torch.div(torch.arange(0, dimension / 2) * 2, float(dimension))
        )
        num = torch.arange(0, max_seq_len).unsqueeze(1)

        positional_enc[:, 0::2], positional_enc[:, 1::2] = (
            torch.sin(num / den),
            torch.cos(num / den),
        )
        positional_enc = positional_enc.unsqueeze(0)
        self.register_buffer("positional_enc", positional_enc)

    def forward(self, input_vec):
        seq_len = input_vec.size(1)
        return self.dropout(input_vec + Variable(self.positional_enc[:, :seq_len]))


class MultiHeadedAttention(nn.Module):
    def __init__(self, dimension, heads, dropout=0.0):
        super().__init__()
        self.heads = heads
        self.dimension = dimension
        self.queryl = nn.Linear(dimension, dimension)
        self.keyl = nn.Linear(dimension, dimension)
        self.valuel = nn.Linear(dimension, dimension)
        self.outl = nn.Linear(dimension, dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):

        assert self.dimension == query.size(-1)
        batch_size = query.size(0)

        query = self.queryl(query)
        key = self.keyl(key)
        value = self.valuel(value)

        query = query.view(
            batch_size, -1, self.heads, query.size(-1) // self.heads
        ).transpose(1, 2)
        key = key.view(
            batch_size, -1, self.heads, key.size(-1) // self.heads
        ).transpose(1, 2)

        value = value.view(
            batch_size, -1, self.heads, value.size(-1) // self.heads
        ).transpose(1, 2)


        attn = self.attention(query, key, value, mask, self.dropout)


        concat = attn.transpose(1, 2).reshape(
            batch_size, -1, query.size(-1) * self.heads
        )

        return self.outl(concat)


    def attention(self, query, key, value, mask=None, dropout=None):
        qk = torch.div(
            torch.matmul(query, key.transpose(-2, -1)), math.sqrt(query.size(-1))
        )

        if mask is not None:
            mask = mask.unsqueeze(1)
            qk = qk.masked_fill(mask == 0, -1e9)

        qk = nn.Softmax(dim=-1)(qk)
        qk = self.dropout(qk) if dropout is not None else qk
        return torch.matmul(qk, value)


class FeedForwardNet(nn.Module):
    def __init__(self, dimension, dff=2048, dropout=0.1):
        super().__init__()
        self.l = nn.Linear(dimension, dff)
        self.out = nn.Linear(dff, dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_vec):
        return self.out(self.dropout(F.relu(self.l(input_vec))))


class LayerNorm(nn.Module):
    def __init__(self, dimension, delta=1e-6):
        super().__init__()
        self.gain = nn.Parameter(torch.ones(dimension))
        self.bias = nn.Parameter(torch.zeros(dimension))
        self.delta = delta

    def forward(self, input_vec):
        mean = torch.mean(input_vec, dim=-1, keepdim=True)
        std = torch.std(input_vec, dim=-1, keepdim=True) + self.delta
        return (self.gain / std) * (input_vec - mean) + self.bias


class ResidualConnection(nn.Module):
    def __init__(self, dimension, dropout=0.1):
        super().__init__()
        self.norm = LayerNorm(dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_vec, sublayer):
        return input_vec + self.dropout(sublayer(self.norm(input_vec)))


def mask(src, trg, padidx):
    src_mask = (src != padidx).unsqueeze(-2)

    if trg is not None:
        trg_mask = (trg != padidx).unsqueeze(-2)
        dimension = trg.size(1)  # get seq_len for matrix

        mask = torch.tril(torch.ones(1, dimension, dimension))
        mask[mask != 0] = 1
        mask = Variable(mask > 0)

        if trg.is_cuda:
            mask.cuda()
        trg_mask = trg_mask & mask

    else:
        trg_mask = None
    return src_mask, trg_mask


class EncoderLayer(nn.Module):
    def __init__(self, dimension, head=8, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadedAttention(dimension, head, dropout)
        self.ffnn = FeedForwardNet(dimension, dropout=dropout)
        self.resconn1 = ResidualConnection(dimension, dropout)
        self.resconn2 = ResidualConnection(dimension, dropout)

        self.norm = LayerNorm(dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_vec, mask=None):
        attn = self.resconn1(input_vec, lambda x: self.attn(x, x, x, mask))
        return self.resconn2(attn, self.ffnn), attn


class Encoder(nn.Module):
    def __init__(
        self, vocab_size, number_of_layers, head, max_seq_len, dimension, dropout, kernel_sizes
    ):
        super().__init__()
        self.emb = EncEmbedding(vocab_size, dimension, dimension, kernel_sizes, dropout)
        self.penc = PositionalEncoding(max_seq_len, dimension, dropout)
        self.enclays = nn.ModuleList(
            [
                copy.deepcopy(EncoderLayer(dimension, head, dropout))
                for _ in range(number_of_layers)
            ]
        )
        self.norm = LayerNorm(dimension)
        self.conv2 = TextCNN(dimension, dimension, kernel_sizes=[1,2], dropout=dropout)

    def forward(self, input_vec, mask=None):
        emb = self.emb(input_vec)
        emb = self.penc(emb)

        for layer in self.enclays:
            emb, _ = layer(emb, mask)
        emb = self.norm(emb)

        seq_len = emb.size(1)
        sentence_embedding = self.conv2(emb, mask)
        sentence_embedding = sentence_embedding.repeat_interleave(seq_len, dim=0).view(emb.size(0),-1,emb.size(-1))
        return sentence_embedding, _


class DecoderLayer(nn.Module):
    def __init__(self, dimension, heads=8, dropout=0.1):
        super().__init__()
        self.ffnn = FeedForwardNet(dimension, dropout=dropout)
        self.resconn = nn.ModuleList(
            [copy.deepcopy(ResidualConnection(dimension, dropout)) for _ in range(3)]
        )
        self.attn = nn.ModuleList(
            [
                copy.deepcopy(MultiHeadedAttention(dimension, heads, dropout))
                for _ in range(2)
            ]
        )

    def forward(self, input_vec, encoder_output, encmask, decmask):
        selfattn = self.resconn[0](input_vec, lambda x: self.attn[0](x, x, x, decmask))

        encdecattn = self.resconn[1](
            selfattn, lambda x: self.attn[1](x, encoder_output, encoder_output, encmask)
        )

        return self.resconn[2](encdecattn, self.ffnn)


class Decoder(nn.Module):
    def __init__(
        self, vocab_size, number_of_layers, head, max_seq_len, dimension, dropout
    ):
        super().__init__()
        self.emb = Embedding(vocab_size, dimension)
        self.penc = PositionalEncoding(max_seq_len, dimension, dropout)
        self.declays = nn.ModuleList(
            [
                copy.deepcopy(DecoderLayer(dimension, head, dropout))
                for i in range(number_of_layers)
            ]
        )
        self.norm = LayerNorm(dimension)

    def forward(self, input_vec, encoder_output, encmask, decmask):

        emb = self.emb(input_vec)
        emb = self.penc(emb)

        for layer in self.declays:
            emb = layer(emb, encoder_output, encmask, decmask)
        return self.norm(emb)


class Transformer(nn.Module):
    def __init__(
        self,
        envocab_size,
        devocab_size,
        kernel_sizes,
        enc_max_seq_len,
        dec_max_seq_len,
        head,
        number_of_layers,
        dimension,
        dropout,
    ):
        super().__init__()
        self.encoder = Encoder(
            envocab_size, number_of_layers, head, enc_max_seq_len, dimension, dropout, kernel_sizes
        )
        self.decoder = Decoder(
            devocab_size, number_of_layers, head, dec_max_seq_len, dimension, dropout
        )
        self.ffnn = nn.Linear(dimension, devocab_size)

    def forward(self, enc_input_vec, dec_input_vec, encmask, decmask):

        encout, _ = self.encoder(enc_input_vec, encmask)
        decout = self.decoder(dec_input_vec, encout, encmask, decmask)

        return F.log_softmax(self.ffnn(decout), dim=-1)


class Batch:
    def __init__(self, src, trg=None, device='cpu', pad=0):
        self.src = src

        src = (src!=pad).any(-1).int()
        self.src_mask = (src!=pad).unsqueeze(1)#.unsqueeze(2)


        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.ntokens = (self.trg_y != pad).data.sum()

            trg_mask = (self.trg != pad).unsqueeze(-2)
            dimension = self.trg.size(-1)

            mask = torch.tril(torch.ones(1, dimension, dimension, device=device))
            mask[mask != 0] = 1
            mask = Variable(mask > 0)

            if self.trg.is_cuda:
                mask.cuda()
            self.trg_mask = trg_mask & mask
 

class LabelSmoothing(nn.Module):
    def __init__(self, vocab_size, pad_index, alpha):
        super().__init__()
        self.alpha = alpha
        self.vocab_size = vocab_size
        self.pad_index = pad_index

    def forward(self, prediction, target):
        prediction = prediction.contiguous().view(-1, prediction.size(-1))
        target = target.contiguous().view(-1)

        one_hot_target = torch.nn.functional.one_hot(target, num_classes=prediction.size(-1))
        one_hot_target[:, self.pad_index] = 0
        one_hot_target = (one_hot_target * (1 - self.alpha)) + (
            self.alpha / (self.vocab_size - 2)
        )
        one_hot_target.masked_fill_((target == self.pad_index).unsqueeze(1), 0)

        return F.kl_div(prediction, one_hot_target, reduction="sum")


class CustomAdam:
    def __init__(self, dimension, optimizer, warmup_steps=4000, step_num=0):
        self.optimizer = optimizer
        self.step_num = step_num
        self.dimension = dimension
        self.warmup_steps = warmup_steps

    def step(self):
        self.step_num += 1
        lr = self.rate()

        for pg in self.optimizer.param_groups:
            pg["lr"] = lr

        self.optimizer.step()

    def rate(self):
        return self.dimension ** (-0.5) * min(
            self.step_num ** (-0.5), self.step_num * self.warmup_steps ** (-1.5)
        )


def data_gen(V, batch, nbatches):
    """
    :param V:
    :type V: int
    :param batch:
    :type batch: int
    :param nbatches:
    :type nbatches: int
    :return:
    """
    # Generate random data for a src-tgt copy task.
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        # src, tgt sizes -> [batch size, seq len]
        yield Batch(src, tgt, 0)
        
        
def init_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [14]:
INPUT_DIM = len(cdata.char2idx)
OUTPUT_DIM = len(cdata.char2idx)
HID_DIM = 512
LAYERS = 2
HEADS = 8
DROPOUT = 0.1
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.05
DEC_DROPOUT = 0.05
SRC_PAD_IDX = 0
TRG_PAD_IDX = 0
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kernel_sizes = [1,2,3,7]

model = Transformer(INPUT_DIM, OUTPUT_DIM, kernel_sizes, cdata.max_sent_len, cdata.max_output_len, HEADS, LAYERS, HID_DIM, DROPOUT).to(device)
model = torch.nn.DataParallel(model)

In [15]:
if os.path.isfile(folder_path + 'checkpoint/scratch/model.pt'):
    checkpoint = torch.load(folder_path + 'checkpoint/scratch/model.pt')
    
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    print('loading the model to train further..')
    
else:
    checkpoint = {'step_num':0, 'epoch':0}
    model.apply(init_weights)
    print('model weights initialised..')

model weights initialised..


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,029,227 trainable parameters


In [17]:
optim = torch.optim.Adam(
            model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9
        )

if 'optimizer_state_dict' in checkpoint:
    optim.load_state_dict(checkpoint['optimizer_state_dict'])

optimizer = CustomAdam(
        dimension=512,
        warmup_steps=400,
        optimizer=optim,
        step_num=checkpoint['step_num'],
    )

In [18]:
criterion = LabelSmoothing(vocab_size=INPUT_DIM, pad_index=0, alpha=0.1)

In [19]:
# tensorboard
from torch.utils.tensorboard import SummaryWriter

shutil.rmtree('/home/qblocks/tensorboard/case_logs_v2/', ignore_errors=True)
writer = SummaryWriter('tensorboard/case_logs_v2')

# # get some random training data
# dataiter = iter(trainloader)
# src, trg = dataiter.next()
# src, trg = src.to(device), trg.to(device)
# batcher = Batch(src, trg, device, 0)
# print('batcher built..')

# # add model to tensorboard
# with SummaryWriter(comment='model') as w:
#     w.add_graph(model, (batcher.src, batcher.trg, batcher.src_mask, batcher.trg_mask,), verbose=True)

In [20]:
from tqdm import tqdm


def train(epoch, model, device, trainloader, optimizer):
    model.train()

    running_loss = 0
    running_accuracy = 0
    
    iterator = 0
    correct = 0
    processed = 0
    pbar = tqdm(trainloader)

    for i, (src, trg) in enumerate(pbar, 0):

        src, trg = src.to(device), trg.to(device)
        batcher = Batch(src, trg, device, 0)

        output = model(batcher.src, batcher.trg, batcher.src_mask, batcher.trg_mask)

        loss = criterion(output, batcher.trg_y) / batcher.ntokens
        loss.backward()

        optimizer.step()
        optimizer.optimizer.zero_grad()

        pred = output.argmax(-1)
        labels = batcher.trg_y

        correct += pred.eq(labels.view_as(pred)).sum().item()
        processed += torch.count_nonzero(labels).item()

        # tensorboard writing every 200 mini-batches
        running_loss += loss.item()
        running_accuracy += round((correct / processed) * 100, 14)

        if i % 200 == 199:
            # log the running loss
            writer.add_scalar(
                "training loss", running_loss / 200, epoch * len(trainloader) + i
            )
            running_loss = 0.0

            # log the running accuracy
            writer.add_scalar(
                "training accuracy",
                running_accuracy / 200,
                epoch * len(trainloader) + i,
            )
            running_accuracy = 0.0

        # tqdm writing
        pbar.set_description(
            desc="Train Epoch - {epoch}, Mini Batch - {batch}, Train Accuracy - {accuracy}, Train Loss - {loss}".format(
                epoch=epoch + 1,
                batch=i + 1,
                accuracy=round((correct / processed) * 100, 14),
                loss=round(loss.item(), 4),
            )
        )

In [21]:
def test(model, device, testloader):
    model.eval()

    iterator = 0
    test_loss = 0
    correct = 0
    processed = 0
    pbar = tqdm(testloader)

    with torch.no_grad():
        for i, (src, trg) in enumerate(pbar, 0):
            src, trg = src.to(device), trg.to(device)
            batcher = Batch(src, trg, device, 0)

            output = model(batcher.src, batcher.trg, batcher.src_mask, batcher.trg_mask)

            loss = criterion(output, batcher.trg_y) / batcher.ntokens
            test_loss += loss

            pred = output.argmax(-1)
            labels = batcher.trg_y

            correct += pred.eq(labels.view_as(pred)).sum().item()
            processed += torch.count_nonzero(labels).item()

            pbar.set_description(
                desc="Test Accuracy - {accuracy}, Test Loss - {loss}".format(
                    accuracy=round((correct / processed) * 100, 14),
                    loss=round(loss.item(), 4),
                )
            )

        test_loss /= len(testloader.dataset)

        # tensorboard writing
        test_accuracy = round((correct / processed) * 100, 14)

        # log the testing loss
        writer.add_scalar("testing loss", test_loss, epoch * len(trainloader) + i)

        # log the testing accuracy
        writer.add_scalar(
            "testing accuracy", test_accuracy, epoch * len(trainloader) + i
        )

        return test_loss

In [22]:
import numpy as np

char2idx = config['char2idx']
idx2char = {v:k for k,v in char2idx.items()}

def pad_nested_sequences(sequences, max_sent_len=4, max_word_len=10, dtype='int32'):
    for sent in sequences:
        max_sent_len = max(len(sent), max_sent_len)
        for word in sent:
            max_word_len = max(len(word), max_word_len)

    x = np.zeros((len(sequences), max_sent_len, max_word_len)).astype(dtype)
    for i, sent in enumerate(sequences):
        for j, word in enumerate(sent):
            if j<max_word_len:
                x[i, j, :len(word)] = word

    return x


def seq2seq(text_list):
    model.eval()

    text = [[[char2idx[char] for char in word] for word in query.split()] for query in text_list]
    text = pad_nested_sequences(text, max_sent_len, max_word_len)
    text = torch.from_numpy(np.array(text)).to(torch.int64).to(device)

    src = (text!=0).any(-1).int()        
    src_mask = (src != 0).unsqueeze(1)

    decoder_inp = [char2idx["<SOS>"]]

    for i in range(max_output_len):
        trg_tensor = torch.LongTensor(decoder_inp).unsqueeze(0).to(device)

        trg_mask = (trg_tensor != 0).unsqueeze(-2)
        dimension = trg_tensor.size(-1)  # get seq_len for matrix

        mask = torch.tril(torch.ones(1, dimension, dimension, device=device))
        mask[mask != 0] = 1
        mask = Variable(mask > 0)

        if trg_tensor.is_cuda:
            mask.cuda()
        trg_mask = trg_mask & mask

        with torch.no_grad():
            output = model.forward(text, trg_tensor, src_mask, trg_mask)

        pred_token = output.argmax(2)[:,-1].item()
        decoder_inp.append(pred_token)

        if pred_token == char2idx["<EOS>"]:
            break

    return ''.join([idx2char[i] for i in decoder_inp][1:])
  
# # 'DETTOL ORIGINAL HAND WASH REFILL PACKET 175 ML RECKITT BENCKISER INDIA LIMITED<EOS>'
# seq2seq(['Reckitt'.upper()])

In [ ]:
# Word ratio maintained in all data
###### FIRST 2 LAYERS ########
# Training data = clean
# epoch 1 - Train Accuracy - 81.9931122560466
# epoch 2 - Train Accuracy - 87.42581597586063
# epoch 3 - Train Accuracy - 88.16240369412816
# epoch 4 - Train Accuracy - 88.5647180347854
# epoch 6 - Train Accuracy - 89.0294778071759
# epoch 20 - Train Accuracy - 90.02246121773504
# epoch 127 - Train Accuracy - 91.0005309363059
# business test accuracy - 94
# train for 150 epochs in general

###### LAST 1/2 LAYERS ########
# Training data = sampled clean + augmentation
# Train Accuracy - 88
# business test accuracy - 98.6

###### LAST 1/2 LAYERS ########
# Training data = sampled clean + augmentation + Feedback
# Train Accuracy - 88
# business test accuracy - 99.5

In [ ]:
from pathlib import Path

EPOCHS = 1000
best_valid_loss = float("inf")

for epoch in range(checkpoint["epoch"]+1, EPOCHS):
    print(seq2seq(["Reckitt".upper()]))
    print(seq2seq(["all out".upper()]))
    print(seq2seq(["MORTEIN".upper()]))

    train(epoch, model, device, trainloader, optimizer)
    test_loss = test(model, device, testloader)

    if test_loss < best_valid_loss:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.optimizer.state_dict(),
                "step_num": optimizer.step_num,
            },
            folder_path + "checkpoint/scratch/model.pt",
        )
    
    if epoch % 5 == 0:
        path_ = folder_path + "checkpoint/scratch/backup/" + str(epoch)
        Path(path_).mkdir(parents=True, exist_ok=True)
        
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.optimizer.state_dict(),
                "step_num": optimizer.step_num,
            },
            path_ + "/model.pt",
        )       

        # best_valid_loss = test_loss

3N5M<SOS>V 5555M<SOS>V55555555555555555555<SOS>V 5555555M<SOS>VF<SOS>VFU<SOS>VFU<SOS>VVFU<SOS>VFFU<SOS>VFFFFFFFFFFFFU<SOS>VVVVV D3 V 555555555555.D3PNU<SOS>V V 55555555555TU<SOS>VFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
3N5.VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV 5VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV VVVV V 5VV 55555V VVVVVV VVVVVVVVVVVVVVVVVVVVVVVVVVV V V 5VVV VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV


  0%|          | 0/5868 [00:00<?, ?it/s]

3NN5.DZN5.DM<SOS>3N5.D3<SOS>3<SOS>3<SOS>3<SOS>3555M<SOS>3<SOS>3<SOS>3<SOS>35M<SOS>VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV V V V V V V V 5555.D3YV V V V V V V VVV V V VVVVVV V V V V V V V V V V V 5.V V VVVVVVVVVVVV V V V VVVVVVVVVV V 55


Train Epoch - 2, Mini Batch - 5868, Train Accuracy - 81.9931122560466, Train Loss - 0.3427: 100%|██████████| 5868/5868 [1:14:21<00:00,  1.32it/s] 
Test Accuracy - 90.67599067599068, Test Loss - 0.287: 100%|██████████| 1/1 [00:00<00:00, 23.52it/s]


DETTOL COOL MINT FLAVOUR SUGAR FREE BOTTLE 100ML RECKITT BENCKISER<EOS>
ALL OUT MULTI FRESH SHAMPOO 100ML 2<EOS>


...

Train Epoch - 153, Mini Batch - 5868, Train Accuracy - 91.07953221328314, Train Loss - 0.2615: 100%|██████████| 5868/5868 [1:14:23<00:00,  1.31it/s]
Test Accuracy - 96.03729603729604, Test Loss - 0.1447: 100%|██████████| 1/1 [00:00<00:00, 24.98it/s]


DETTOL COOL SOAP 75GM RECKITT BENCKISER<EOS>
ALL OUT ULTRA MOSQUITO REPELLENT REPELLENT REFILL 7.5ML<EOS>


  0%|          | 0/5868 [00:00<?, ?it/s]

MORTEIN INSTANT REFILL PACKET 1<EOS>


Train Epoch - 154, Mini Batch - 2142, Train Accuracy - 91.0786598432008, Train Loss - 0.2483:  37%|███▋      | 2142/5868 [27:16<48:09,  1.29it/s]   Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/qblocks/.local/lib/python3.8/site-packages/tensorboard/summary/writer/event_file_writer.py", line 252, in run
    self._record_writer.flush()
  File "/home/qblocks/.local/lib/python3.8/site-packages/tensorboard/summary/writer/record_writer.py", line 43, in flush
    self._writer.flush()
  File "/home/qblocks/.local/lib/python3.8/site-packages/tensorflow/python/lib/io/file_io.py", line 222, in flush
    self._writable_file.flush()
tensorflow.python.framework.errors_impl.PermissionDeniedError: tensorboard/case_logs_v2/events.out.tfevents.1629005675.pharmeasy3090.303.0; Read-only file system
Train Epoch - 154, Mini Batch - 3199, Train Accuracy - 91.07761917028489, Train Loss - 0.2

In [ ]:
# try examples and explore - v1 Co trimoxazole
query = ['oxi meter'.upper()]
xx = [[corpus[0][v], corpus[2][v]] for v in __similar_topn__(np.array(infer(query)[0][1].cpu()), np.array(corpus[1]), 1000)][:300]
xx

In [1]:
!nvidia-smi

Sun Aug 15 05:31:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    On   | 00000000:19:00.0 Off |                  N/A |
| 53%   55C    P8    21W / 275W |      0MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    On   | 00000000:1A:00.0 Off |                  N/A |
| 47%   

In [ ]:
# oxy meter

In [22]:
# (first 100 clean | 0 rest | no feedback), then (100 clean | 2 rest | feedback), then (100 clean | 0 rest | feedback)

Wed Mar 17 18:35:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    On   | 00000000:19:00.0 Off |                  N/A |
|  0%   56C    P8    18W / 300W |  24151MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    On   | 00000000:1A:00.0 Off |                  N/A |
| 31%   

In [ ]:
from shutil import copyfile


def save_checkpoint(checkpoint_name):
    global model, config
    
    init_path = './checkpoint/'
    checkpoint_path = init_path + checkpoint_name
    
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
    
    # model
    torch.save(model.state_dict(), checkpoint_path + '/model.pt')
    
    # configuration
    with open(checkpoint_path + '/config.pkl', 'wb') as fp:
        pickle.dump(config, fp)
    
    # training data
    copyfile(training_data_path, checkpoint_path+'/training_data.pkl')

In [22]:
# 88.253

In [ ]:
# base catalog -> product level
# output -> has less attributes
# testing -> on same data

# ['ucode', 'catalog']

In [ ]:
# base catalog -> attribute level
# output -> good number of attributes
# testing -> on same data

# ['ucode', 'catalog mdm and rest']